## Rust Ownership and Borrowing

## Problems

In [4]:
let a = 10;
let b = a;
println!("{}", a); // 10
println!("{}", b); // 10

10
10


In [7]:
let a = String::from("Hello");
let b = a; // the value of a is moved to b, a is no longer valid
println!("{}", a); // error 
println!("{}", b); 

Error: borrow of moved value: `a`

Two example above yield different results because `int` implements the `Copy` trait, while `String` does not.

Objects which implement `Copy` traits are copied when they are assigned to another variable. Objects which do not implement `Copy` trait are **moved** when they are assigned to another variable.

In [8]:
let a = 10;
let b = a;

// print the address of a and b
println!("{:p}", &a);
println!("{:p}", &b); 

0x16b856810


0x16b856814


They are different objects. To achieve the same result as the first example, we can use the `clone` method to create a new object.

In [9]:
let a = String::from("Hello");
let b = a.clone();

println!("{}", a); // Hello
println!("{}", b); // Hello

// print the address of a and b
println!("{:p}", &a);
println!("{:p}", &b);

Hello
Hello
0x16b8567e0
0x16b8567f8


## Copy vs Clone

`Copy` is a special trait that is used for types that can be copied by simply copying bits. This is mainly used for simple types like integers, floats, and booleans. If a type implements the `Copy` trait, an older variable is still usable after assignment.

While `Clone` trait is used for types that cannot be copied by simply copying bits. If a type implements the `Clone` trait, we can create a new object by cloning the original object. This often involves allocating memory on the heap and deep copying the original object.

From now on, our focus will be on non-`Copy` types.

## Stack vs Heap

Stack and heap are two different memory regions in a program. 

### Stack
- is used for static memory allocation
- is used to store local variables and function call information 
- the size must be known at compile time
- is faster than heap memory
- when a function is called, a block of memory is allocated on the stack for the function to use
- when a function call ends, the block of memory is deallocated

### Heap
- is used for dynamic memory allocation
- is used to store data whose size is not known at compile time
- is slower than stack memory because it is allocated at runtime
- when a block of memory is allocated on the heap, a pointer to that memory is returned
- to access the data in the heap, we need to follow the pointer


## Ownership Rules

- Each value in Rust has a variable that is its owner.
- There can only be one owner at a time.
- When the owner goes out of scope, the value will be dropped.

In [11]:
let x = String::from("Hello");
let y = x; // the owner is transferred to y
// x no longer valid

println!("{}", x); 

Error: borrow of moved value: `x`

In [13]:
fn uppercase(s: String) -> String {
    s.to_uppercase()
}

let x = String::from("Hello");
println!("{}", uppercase(x)); // the value of x is moved to the function parameter

println!("{}", x); // error

Error: borrow of moved value: `x`

Wow, so many errors! Let's fix them one by one.

The easiest way to fix the errors is to clone the `String` object. This way, we can create a new object on the heap and assign it to the new variable.

In [14]:
let x = String::from("Hello");
let y = x.clone();

println!("{}", x); // Hello
println!("{}", y); // Hello

Hello
Hello


In [15]:
fn uppercase(s: String) -> String {
    s.to_uppercase()
}

let x = String::from("Hello");
println!("{}", uppercase(x.clone())); // clone the value of x

println!("{}", x); // Hello

HELLO
Hello


Solved! 

But wait, we cloned the object! There are 2 problems:

- We are creating a new object on the heap, which is slower than copying bits.
- We are using more memory than necessary.


## Borrowing

We can do better by using `borrowing`. Borrowing allows us to pass a reference to the object instead of passing the object itself. This way, we can avoid creating a new object on the heap.

In [19]:
fn test() {
    let x = String::from("Hello");
    let y = &x;

    println!("{}", x); // Hello
    println!("{}", *y); // Hello
}

test();

Hello
Hello


In [20]:
fn uppercase(s: &String) -> String {
    s.to_uppercase()
}

let x = String::from("Hello");
println!("{}", uppercase(&x));

println!("{}", x); // Hello

HELLO
Hello


## Forward

In [22]:
fn uppercase(s: String) -> (String, String) {
    let result = s.to_uppercase();
    (result, s)
}

let x = String::from("Hello");
let (upper, x) = uppercase(x);

println!("{}", upper); // HELLO
println!("{}", x); // Hello

HELLO
Hello


## Mutable Reference

In [25]:
fn change_to_upper(s: &String) {
    s.make_ascii_uppercase()
}

let x = String::from("Hello");
change_to_upper(&x);

println!("{}", x); // error

Error: cannot borrow `*s` as mutable, as it is behind a `&` reference

In [26]:
fn change_to_upper(s: &mut String) {
    s.make_ascii_uppercase()
}

let mut x = String::from("Hello");
change_to_upper(&mut x);

println!("{}", x); // HELLO

HELLO


## Struct